# AAI 594 — Assignment 3

## Building Agent Tools

**In this lab you will:**
- **Required (Sections 1–5):** Create **Unity Catalog function tools** — one SQL function and one Python function — and test them.
- **Required (Section 6):** Set up **semantic search with embeddings** on the UltraFeedback dataset so an agent can find similar instructions by meaning.
- **Required (Section 7):** Configure an **external MCP server** (You.com web search) in Databricks so your agent can access live web information.
- **Optional, strongly encouraged (Section 8):** Create an **Agent Skill** (`SKILL.md`) that documents the tools you built.

### The big picture

Over Weeks 3–5 you are building an **UltraFeedback Expert** agent — an AI assistant that helps users explore and understand LLM preference data. This week you create the **tools**; next week you wire them into a working agent; in Week 5 you evaluate how well it performs.

| Week | What you do | Deliverable |
|------|------------|-------------|
| 3 (this week) | Build tools: UC functions, semantic search, MCP | Tested tools + MCP config |
| 4 | Wire tools into an agent; register a prompt; compare LLMs | Working agent |
| 5 | Evaluate the agent with judges and an eval dataset | Evaluation report |

**Readings this week:**
- [Practical Guide for Agentic AI Workflows](https://arxiv.org/pdf/2512.08769)
- [MCP Architecture](https://modelcontextprotocol.io/docs/learn/architecture)

**Key docs:**
- [Create AI agent tools with UC functions](https://docs.databricks.com/aws/en/generative-ai/agent-framework/create-custom-tool)
- [Foundation Model APIs — Embeddings](https://docs.databricks.com/en/machine-learning/model-serving/score-foundation-models.html)
- [External MCP Servers on Databricks](https://docs.databricks.com/aws/en/generative-ai/mcp/external-mcp)
- [You.com on Databricks Marketplace](https://marketplace.databricks.com/details/32d5b7b6-0fab-4bba-9c57-5de23dd58996/Youcom_Youcom-MCP-The-1-AI-Web-Search-API)

---
## 1. Why agents need tools *(Required)*

An LLM on its own can only generate text. **Tools** give agents the ability to *act* — query databases, search the web, look up facts, run computations. In this assignment you'll create three kinds of tools:

| Tool type | What it does | Example |
|-----------|-------------|--------|
| **UC SQL function** | Deterministic lookup against structured data | "How many rows come from `evol_instruct`?" |
| **UC Python function** | Custom computation or text processing | "Analyze the complexity of this instruction" |
| **Semantic search** | Embedding-based similarity search over text | "Find instructions similar to *Explain quantum tunneling*" |
| **External MCP** | Access external services (web search, APIs) | "Search the web for recent LLM benchmarks" |

Each tool is registered in a place the agent can discover it — Unity Catalog for functions and semantic search, MCP for external services.

---
## 2. Install dependencies *(Required)*

We need two packages:
- `unitycatalog-ai[databricks]` — the Unity Catalog AI client for creating and testing UC functions as agent tools.
- `numpy` — for computing cosine similarity between embedding vectors.

**Docs:** [Unity Catalog AI](https://docs.unitycatalog.io/ai/) · [Foundation Model APIs](https://docs.databricks.com/en/machine-learning/model-serving/score-foundation-models.html)

In [ ]:
# Install the UC AI client (for creating/testing UC functions as tools)
# and numpy (for computing cosine similarity in semantic search)
%pip install unitycatalog-ai[databricks] numpy
dbutils.library.restartPython()

---
## 3. Verify your data *(Required)*

Confirm the UltraFeedback table from Assignment 1 is still available. If you get an error, re-run Assignment 1 first.

In [ ]:
# Quick check: confirm the table exists, show schema and row count
df = spark.table("main.default.assignment_file")
print(f"Row count: {df.count():,}")
print(f"Columns:  {df.columns}")
df.printSchema()
display(df.limit(3))

---
## 4. Create Unity Catalog function tools *(Required)*

Unity Catalog functions are UDFs registered in `catalog.schema.function_name`. When an agent needs a tool, it calls the function by name. Two patterns are common:

1. **SQL functions** — best for deterministic lookups against tables (e.g., counts, filters, joins).
2. **Python functions** — best for custom logic, text processing, or computations that don't map cleanly to SQL.

You'll create two SQL functions, a Python function, then build your own.

These functions exist in Unity Catalog, so you can see them in the UI after registration

**Docs:** [Create AI agent tools with UC functions](https://docs.databricks.com/aws/en/generative-ai/agent-framework/create-custom-tool) · [CREATE FUNCTION syntax](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-ddl-create-sql-function)

### 4.1 UC SQL functions

#### all_model_combinations

This function shows all combinations of chosen & rejected models, sorted by the most common combinations.

**Key points:**
- The `COMMENT` on the function and its parameters helps the agent understand *when* and *how* to use the tool. Write clear, descriptive comments.
- The function returns a `TABLE` which has all the data in the query returned

In [ ]:
%sql
CREATE OR REPLACE FUNCTION main.default.all_model_combinations()
RETURNS TABLE
LANGUAGE SQL
COMMENT 'Shows all combinations of chosen and rejected models, counts and average ratings'
RETURN (
  SELECT 
    `chosen-model`,
    `rejected-model`,
    count(source) as records,
    avg(`chosen-rating`) as avg_chosen_rating,
    avg(`rejected-rating`) as avg_rejected_rating
  FROM main.default.assignment_file
  GROUP BY 1,2
  ORDER BY 3 desc
)

In [ ]:
%sql
select * from main.default.all_model_combinations() limit 10

#### compare_models

The function below compares two specific models to see how frequently an individual model wins vs. loses.

This also returns a table with the count of each scenario, as well as the average winning and losing rating

In [ ]:
%sql
CREATE OR REPLACE FUNCTION main.default.compare_models(
  model_a STRING COMMENT 'First model name to compare e.g. gpt-4',
  model_b STRING COMMENT 'Second model name to compare'
)
RETURNS TABLE
LANGUAGE SQL
COMMENT 'Compares two models by how often each was chosen vs rejected in the UltraFeedback dataset. Returns a summary string with win counts for each model.'
RETURN (
  SELECT 
    CASE when `chosen-model` = model_a AND `rejected-model` = model_b THEN 'A_win'
         when `chosen-model` = model_b AND `rejected-model` = model_a THEN 'B_win'
         else 'other' end as comparison_scenario,
    count(*) as win_count,
    avg(`chosen-rating`) as avg_chosen_rating,
    avg(`rejected-rating`) as avg_rejected_rating
  FROM main.default.assignment_file
  WHERE `chosen-model` IN (model_a, model_b)
     AND `rejected-model` IN (model_a, model_b)
  GROUP BY 1
  ORDER BY 1
)

In [ ]:
%sql
select * 
from main.default.compare_models('gpt-3.5-turbo', 'alpaca-7b')

### 4.2 Python function: `analyze_instruction`

This function takes an instruction text and returns complexity metrics (word count, sentence count, estimated complexity level). An agent could use this to assess how complex a prompt is before deciding how to handle it.

**Key points:**
- Python UC functions must have **type hints** on all arguments and the return value.
- **Imports go inside the function body** — they won't be resolved otherwise.
- Use [Google-style docstrings](https://google.github.io/styleguide/pyguide.html#383-functions-and-methods) so the agent can parse the description.

In [ ]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

# Initialize the Databricks Function Client
uc_client = DatabricksFunctionClient()

# Define the Python function with type hints and a clear docstring.
# NOTE: all imports must be INSIDE the function body.
def analyze_instruction(instruction: str) -> str:
    """
    Analyzes the complexity and characteristics of an instruction prompt.

    Returns word count, sentence count, average word length, estimated
    complexity level (low/medium/high), and whether the text is a question.
    Use this to assess instruction difficulty before generating a response.

    Args:
        instruction: The instruction or prompt text to analyze.

    Returns:
        A JSON string with analysis metrics.
    """
    import json
    import re

    words = instruction.split()
    word_count = len(words)
    sentences = [s.strip() for s in re.split(r'[.!?]+', instruction) if s.strip()]
    sentence_count = len(sentences)
    avg_word_length = round(sum(len(w) for w in words) / max(word_count, 1), 1)
    is_question = instruction.strip().endswith('?')

    if word_count > 50 or sentence_count > 3:
        complexity = "high"
    elif word_count > 20:
        complexity = "medium"
    else:
        complexity = "low"

    return json.dumps({
        "word_count": word_count,
        "sentence_count": sentence_count,
        "avg_word_length": avg_word_length,
        "complexity": complexity,
        "is_question": is_question
    })

# Register the function in Unity Catalog (main.default schema)
function_info = uc_client.create_python_function(
    func=analyze_instruction,
    catalog="main",
    schema="default",
    replace=True  # overwrite if it already exists
)
print(f"Registered: {function_info.full_name}")

In [ ]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

uc_client = DatabricksFunctionClient()

# Test the function out that we just created

result = uc_client.execute_function(
    function_name="main.default.analyze_instruction",
    parameters={"instruction": "Heisenberg explained resonance theory, which is a structure-activity relationship that describes the effect of complementary chemical groups on a drug's properties and actions (such as solubility, absorption, and therapeutic effects). These complementary groups can be used to reduce toxicity and increase the potency of drug compounds"})

print(result.value)

%md
### 4.3 Your turn: create a function *(Required)*

Create **two additional UC function** (SQL or Python or both) that would be useful for the UltraFeedback Expert agent. Some  ideas:

| Idea | Type | What it does |x
|------|------|--------------|
| `count_model_appearances` | SQL | Count how often a model appears as chosen vs. rejected |
| `get_sample_pairs` | SQL | Return N example chosen/rejected pairs for a given source |
| `format_comparison` | Python | Take a chosen and rejected response and format them side-by-side |
| `extract_keywords` | Python | Pull key terms from an instruction for categorization |

Make sure your function has:
- A clear `COMMENT` (SQL) or docstring (Python) explaining what it does and when to use it
- Type hints (Python) or typed parameters (SQL)
- A test cell showing it works just like the examples above

In [ ]:
# CREATE YOUR FUNCTIONS HERE
# Use either %%sql for a SQL function or Python with uc_client.create_python_function()
# Then add a test cell below to verify it works.

In [ ]:
# TEST YOUR FUNCTIONS HERE

---
## 5. List your registered tools

Before moving on, verify all your UC functions are registered. The cell below lists functions in `main.default`.

In [ ]:
%%sql
USE CATALOG main;
SHOW USER FUNCTIONS IN main.default;

## 6. Semantic Search with Embeddings *(Required)*

Semantic search lets an agent find **similar text by meaning** — not just exact keyword matches. For the UltraFeedback Expert, this means the agent can find instructions similar to a user's question, even if the wording is different.

**How it works:**
1. **Embed** each instruction using a Foundation Model embedding endpoint (`databricks-gte-large-en`).
2. **Store** the embeddings in a Delta table.
3. At query time, embed the user's question and compute **cosine similarity** against all stored embeddings.

This approach uses the Foundation Model API (available on Free Edition) instead of Vector Search endpoints (which require a quota not available on Free Edition).

> **Why not Vector Search?** Databricks Vector Search requires provisioned endpoints that exceed the Free Edition quota. The manual embedding approach teaches the same concepts (embeddings, similarity search, retrieval) and works with no extra resources.

**Docs:** [Foundation Model APIs — Embeddings](https://docs.databricks.com/en/machine-learning/model-serving/score-foundation-models.html)

### 6.1 Prepare a source table

We'll create a focused table with 100 unique instructions. This keeps embedding costs low while giving the agent plenty of material to search.

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

# First, check what columns are in the source table
af = spark.table("main.default.assignment_file")
print(f"assignment_file columns: {af.columns}")

# Detect the instruction column name (some versions use 'instruction', others use 'prompt')
if "instruction" in af.columns:
    text_col = "instruction"
elif "prompt" in af.columns:
    text_col = "prompt"
else:
    raise ValueError(f"Expected 'instruction' or 'prompt' column. Found: {af.columns}")

print(f"Using text column: '{text_col}'")

# Create a table of 100 unique instructions for semantic search
source_df = (
    af.select("source", af[text_col].alias("instruction"))
    .dropDuplicates(["instruction"])
    .limit(100)
    .withColumn("id", monotonically_increasing_id())
)

source_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("main.default.ultrafeedback_embeddings")

print(f"Created table with {spark.table('main.default.ultrafeedback_embeddings').count()} rows.")
display(spark.table("main.default.ultrafeedback_embeddings").limit(3))


### 6.2 Compute embeddings

We'll use the `databricks-gte-large-en` Foundation Model endpoint to compute embeddings. The endpoint accepts batches of text and returns vectors (1024 dimensions for GTE-Large).

We embed in batches because the API has input size limits.

In [ ]:
import mlflow.deployments
import json
import time

client = mlflow.deployments.get_deploy_client("databricks")

def get_embeddings_batch(texts, endpoint="databricks-gte-large-en", max_retries=5):
    """Embed a list of texts with retry logic for rate limits."""
    for attempt in range(max_retries):
        try:
            response = client.predict(
                endpoint=endpoint,
                inputs={"input": texts}
            )
            return [item["embedding"] for item in response["data"]]
        except Exception as e:
            if "429" in str(e) or "RATE" in str(e).upper():
                wait = 2 ** attempt * 5  # 5s, 10s, 20s, 40s, 80s
                print(f"  Rate limited. Waiting {wait}s (attempt {attempt+1}/{max_retries})...")
                time.sleep(wait)
            else:
                raise e
    raise Exception("Max retries exceeded for embedding request.")

# Load instructions from the source table
instructions_df = spark.table("main.default.ultrafeedback_embeddings").toPandas()
instructions = instructions_df["instruction"].tolist()
ids = instructions_df["id"].tolist()

print(f"Embedding {len(instructions)} instructions (with rate-limit handling)...")

# Embed in small batches with a pause between each to avoid rate limits
BATCH_SIZE = 5
SLEEP_BETWEEN = 2  # seconds between batches
all_embeddings = []

for i in range(0, len(instructions), BATCH_SIZE):
    batch = instructions[i:i + BATCH_SIZE]
    batch_embeddings = get_embeddings_batch(batch)
    all_embeddings.extend(batch_embeddings)
    print(f"  Embedded {min(i + BATCH_SIZE, len(instructions))}/{len(instructions)}")
    if i + BATCH_SIZE < len(instructions):
        time.sleep(SLEEP_BETWEEN)

print(f"Done. {len(all_embeddings)} embeddings, {len(all_embeddings[0])} dimensions each.")


In [ ]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, LongType, StringType, ArrayType, FloatType

# Build a DataFrame with id, instruction, source, and embedding
embedding_records = []
for idx, (row_id, instr, emb) in enumerate(zip(ids, instructions, all_embeddings)):
    embedding_records.append({
        "id": int(row_id),
        "instruction": instr,
        "source": instructions_df.iloc[idx]["source"],
        "embedding": emb
    })

schema = StructType([
    StructField("id", LongType(), False),
    StructField("instruction", StringType(), False),
    StructField("source", StringType(), True),
    StructField("embedding", ArrayType(FloatType()), False),
])

emb_spark_df = spark.createDataFrame(embedding_records, schema=schema)

# Save to Delta — this is our "vector index"
emb_spark_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("main.default.ultrafeedback_embeddings")

print(f"Saved {len(embedding_records)} embeddings to main.default.ultrafeedback_embeddings.")
display(spark.table("main.default.ultrafeedback_embeddings").select("id", "instruction", "source").limit(3))

### 6.3 Create a similarity search function

Now we create a **SQL** UC function that the agent can call. Given a query, it uses keyword matching
(`LIKE`) against stored instructions and returns the top matches.

> **Why SQL instead of Python?** UC Python functions run in an isolated sandbox with no access to
> SparkSession or Databricks auth. SQL functions, by contrast, can natively query Delta tables and
> are executed by the SQL engine — no sandbox restrictions.

In [ ]:
%sql
CREATE OR REPLACE FUNCTION main.default.search_similar_instruct(
  query STRING COMMENT 'The search query — a word, phrase, or topic to find similar instructions in the UltraFeedback dataset. Examples: python, quantum computing, write a story.',
  top_k INT COMMENT 'Maximum number of matching instructions to return. Use 3 for a quick look or 5 for more detail.'
)
RETURNS STRING
COMMENT 'Searches UltraFeedback instructions by keyword matching and returns matching instructions with their source names. Use this to explore what kinds of prompts exist in the dataset for a given topic.'
RETURN (
  SELECT CONCAT(
    'Query: ', query, ' | Showing up to ', CAST(top_k AS STRING), ' matches:\n',
    COALESCE(
      ARRAY_JOIN(
        SLICE(
          COLLECT_LIST(CONCAT('[', source, '] ', LEFT(instruction, 200))),
          1, top_k
        ),
        '\n'
      ),
      'No matches found.'
    )
  )
  FROM main.default.ultrafeedback_embeddings
  WHERE LOWER(instruction) LIKE CONCAT('%', LOWER(query), '%')
)

### 6.4 Test the similarity search

In [ ]:
%sql
-- Test: find instructions about Python
SELECT main.default.search_similar_instruct('python', 3) AS result
UNION ALL
SELECT main.default.search_similar_instruct('quantum', 3)
UNION ALL
SELECT main.default.search_similar_instruct('health benefits', 3)


---
## 7. Configure an external MCP server in Databricks *(Required)*

The **Model Context Protocol (MCP)** is an open standard that lets AI agents connect to external tools and data sources. Databricks supports **external MCP servers** through managed proxy endpoints — you install them from the **Databricks Marketplace** and they become available in **AI Playground** and in your agent code.

You'll install the **You.com MCP server**, which provides tools for:
- **Web search** — search the web, news, and AI-optimized results
- **Content extraction** — extract page content from URLs in markdown format

This means your agent will be able to search the web for current information about LLMs, benchmarks, and research papers — something it can't do with just the UltraFeedback dataset.

**Docs:** [External MCP Servers on Databricks](https://docs.databricks.com/aws/en/generative-ai/mcp/external-mcp) · [You.com on Databricks Marketplace](https://marketplace.databricks.com/details/32d5b7b6-0fab-4bba-9c57-5de23dd58996/Youcom_Youcom-MCP-The-1-AI-Web-Search-API) · [MCP Architecture](https://modelcontextprotocol.io/docs/learn/architecture)

### 7.1 Get a You.com API key

1. Go to [you.com/platform](https://you.com/platform).
2. Sign in or create an account.
3. Generate an API key and copy it. **Keep it safe — you'll need it in the next step.**

- Note you get $100 of complimentary credits

### 7.2 Install You.com MCP server from the Databricks Marketplace

Databricks Marketplace offers curated MCP servers that you can install with a few clicks. This creates a **Unity Catalog connection** that proxies requests to the external MCP server.

#### Steps

1. In your Databricks workspace, go to **Marketplace** → **Agents** → **MCP Servers** tab.
2. Find **You.com** (or search for it) and click **Install**.
3. In the installation dialog:
   - **Connection name**: Enter a name like `youcom_connection` (this becomes the identifier in Unity Catalog).
   - **Host / Base path**: These are pre-populated for Marketplace servers.
   - **Bearer token**: Paste your You.com API key from step 7.1.
4. Click **Install** to create the connection.

Once installed:
- A Unity Catalog connection is created with your MCP server details.
- Databricks provisions a managed proxy endpoint for secure, authenticated access.
- The proxy URL will be: `https://<workspace-hostname>/api/2.0/mcp/external/youcom_connection`

> **Tip:** To view your installed MCP servers, go to your workspace → **Agents** → **MCP Servers**.

> **Alternative (advanced):** You can also create a custom HTTP connection manually. See [External MCP Servers docs](https://docs.databricks.com/aws/en/generative-ai/mcp/external-mcp) for details.

### 7.3 Test your MCP connection in AI Playground

The fastest way to verify your MCP server is working is through **AI Playground**:

1. Go to **AI Playground** in your Databricks workspace.
2. Choose a model with the **Tools enabled** label (e.g., `databricks-meta-llama-3-3-70b-instruct`).
3. Click **Tools** → **+ Add tool** → **MCP Servers** → **External MCP servers**.
4. Select your `youcom_connection` (or whatever you named it).
5. Try a query that requires live web search:
   - *"Search the web for the latest LLM benchmarks from 2025-2026."*
   - *"What are the top open-source LLMs released in the last 6 months?"*

The model should invoke the You.com search tool and return live web results.

> **Take a screenshot** of the agent using the You.com MCP tool in AI Playground. Include it in your submission as `screenshots/mcp_you_com.png`.

> **Troubleshooting:**
> - If the MCP server doesn't appear, verify your connection was created under **Catalog** → **Connections**.
> - Check that your You.com API key is active at [you.com/platform](https://you.com/platform).
> - Ensure you have `USE CONNECTION` privilege on the connection (workspace admins have this by default).

### 7.4 Use the MCP server programmatically *(Preview)*

Once installed, you can use the MCP server in agent code via the Databricks MCP Client:

```python
from databricks.sdk import WorkspaceClient
from databricks_mcp import DatabricksMCPClient

ws = WorkspaceClient()
host = ws.config.host

mcp_client = DatabricksMCPClient(
    server_url=f"{host}/api/2.0/mcp/external/youcom_connection",
    workspace_client=ws
)

# List available tools
tools = mcp_client.list_tools()
print([tool.name for tool in tools])

# Call a search tool
response = mcp_client.call_tool("you-search", {"query": "latest LLM benchmarks 2026"})
print(response.content[0].text)
```

You can also include external MCP servers alongside Databricks managed MCP servers:

```python
MANAGED_MCP_SERVER_URLS = [
    f"{host}/api/2.0/mcp/functions/system/ai",          # Default managed MCP
    f"{host}/api/2.0/mcp/external/youcom_connection"     # External MCP proxy
]
```

We'll use this pattern in later assignments to give agents both UC tools and web search capabilities.

#### Other MCP servers on the Marketplace

| MCP Server | What it provides | Where to find it |
|------------|-----------------|------------------|
| **You.com** (required) | Web search, content extraction | [Marketplace](https://marketplace.databricks.com/details/32d5b7b6-0fab-4bba-9c57-5de23dd58996/Youcom_Youcom-MCP-The-1-AI-Web-Search-API) |
| **GitHub** | Code search, issues, PRs | Marketplace → Agents → MCP Servers |
| **Custom HTTP** | Any MCP server with Streamable HTTP | Manual connection setup |

Browse the full catalog at **Marketplace** → **Agents** → **MCP Servers**, or create custom HTTP connections for self-hosted servers. See the [External MCP docs](https://docs.databricks.com/aws/en/generative-ai/mcp/external-mcp) for details.

---
## 8. Bonus: Create an Agent Skill *(Optional, strongly encouraged)*

An **Agent Skill** is a markdown document (`SKILL.md`) that gives an AI assistant domain knowledge. When a skill is loaded, the assistant knows how to use specific tools, follow procedures, and avoid common mistakes — without you having to explain everything in every prompt.

Think of it as a **user manual for your agent's tools**, written so another AI can follow it.

### Why this matters

You've just built several tools (UC functions, semantic search, MCP). But an AI assistant doesn't automatically know *when* to use each one, *how* to call them, or *what to watch out for*. A skill bridges that gap.

### Create a skill

Create a file called `SKILL.md` in your `assignment_3/` folder with the following structure:

```markdown
---
name: ultrafeedback-expert
description: >
  Tools and knowledge for exploring the UltraFeedback LLM preference dataset.
  Activate when: user asks about LLM preferences, model comparisons, or
  instruction quality in the UltraFeedback dataset.
---

# UltraFeedback Expert

## When to Use This Skill

**Trigger patterns:**
- "UltraFeedback" or "preference data" or "chosen vs rejected"
- "Which model is preferred" or "model comparison"
- "Find similar instructions" or "semantic search"

## Available Tools

| Tool | Type | What it does |
|------|------|--------------|
| `main.default.lookup_source_info` | UC SQL | Returns row count and sample for a source |
| `main.default.analyze_instruction` | UC Python | Analyzes instruction complexity |
| `main.default.compare_models` | UC SQL | Compares two models by chosen vs rejected counts |
| `main.default.get_model_win_rate` | UC SQL | Returns a model's win rate in the dataset |
| `main.default.classify_instruction_topic` | UC Python | Classifies instruction topic by keywords |
| `main.default.search_similar_instructions` | UC SQL | Keyword-based search over dataset instructions |
| You.com MCP (Databricks) | External MCP | Live web search via Databricks proxy |

## Procedures

### Answering "What sources are in the dataset?"
1. Call `lookup_source_info` for each known source.
2. Summarize counts and sample instructions.

### Finding similar instructions
1. Call `search_similar_instructions` with the user's text.
2. Return the top 3-5 matches with their sources and similarity scores.

## Gotchas
- Embeddings table (`main.default.ultrafeedback_embeddings`) must exist with pre-computed vectors.
- Column names with hyphens (e.g., `chosen-model`) need backtick escaping.
```

Fill in the details based on the actual tools you created. You can use this skill in Cursor by placing it in `~/.cursor/skills/` or referencing it in a project rule.

**Docs:** [Agent Skills standard](https://github.com/xnano-ai/agentskills) · [Cursor Rules](https://cursor.com/docs/context/rules) · [Anthropic Agent Skills Guide](https://resources.anthropic.com/hubfs/The-Complete-Guide-to-Building-Skill-for-Claude.pdf?hsLang=en)

---
## Lab complete

### Required (Sections 1–7)
- [ ] **Section 3:** Verified the UltraFeedback table exists.
- [ ] **Section 4:** Created and tested the SQL function (`lookup_source_info`) and Python function (`analyze_instruction`).
- [ ] **Section 4.3:** Created and tested your own UC function.
- [ ] **Section 5:** Listed all registered UC functions.
- [ ] **Section 6:** Created embeddings for 100 instructions, registered the  UC function, and tested semantic search.
- [ ] **Section 7:** Installed the You.com MCP server from Databricks Marketplace and tested it in AI Playground (screenshot taken).

### Optional but strongly encouraged (Section 8)
- [ ] **Section 8:** Created a `SKILL.md` documenting your tools.

**Submit:** Your executed notebook (`.ipynb` with all outputs) and the completed `SUBMISSION_3.md`. Include screenshots in the `screenshots/` folder.

*Next week you'll wire these tools into a working agent, register a prompt, and compare different LLMs.*